<a href="https://colab.research.google.com/github/faezehmontazeri/IPM-AISummer2023/blob/main/Sohrevardi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.13.0


In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()


In [ ]:
#from __future__ import print_function, absolute_import, division, unicode_literals

#import appdirs
#import argparse
#import getpass
#import hashlib
#import json
#import os
#import requests
#import shutil
#import stat

#from builtins import input

import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
#import tensorflow.contrib.eager as tfe

#Eager execution is enabled by default in version 2.x You can check that by using tf.executing_eagerly()
#It should return True.
#If you are having version less then 2.0 then it can be enabled by using tf.enable_eager_execution()
tf.executing_eagerly()
#Eager enabled by default in tf2, you do can disable it as below
#tf.compat.v1.disable_eager_execution()

#!wget https://www.cityscapes-dataset.com/file-handling/?packageID=1
#file_id = '1-7pEZa3O5UTDjH3L6F4ZZBjxMnRgpwEL'
#url = 'https://drive.google.com/uc?id={}'.format(file_id)

#!wget 'https://drive.google.com/open?id=1-7pEZa3O5UTDjH3L6F4ZZBjxMnRgpwEL&authuser=0'
#'https://drive.google.com/file/d/1-7pEZa3O5UTDjH3L6F4ZZBjxMnRgpwEL'
#https://doc-0g-4g-docs.googleusercontent.com/docs/securesc/rgc31q317spmabsa7dm8q0te1a943j7q/ph2rh8qrql13hoadj9jsvupvpp1ro7av/1694785575000/13870146594497830143/10510790426494910345Z/1-7pEZa3O5UTDjH3L6F4ZZBjxMnRgpwEL?e=download&uuid=c34f927e-e26e-42fb-bb89-20b985ad2b0e&nonce=va8235fjkja3u&user=10510790426494910345Z&hash=ohpdppm05n9t0ai81rknh8med4q2tupl
from google.colab import drive
drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [ ]:
!pwd
url = 'gdrive/MyDrive/ColabNotebooks/gtFine_trainvaltest.zip'
!unzip 'gdrive/MyDrive/ColabNotebooks/gtFine_trainvaltest.zip'
#gtFine_trainvaltest.zip

In [ ]:
!pip install cityscapesscripts

In [ ]:
!pip install cityscapesscripts[gui]

In [34]:
import tensorflow_datasets as tfds
ds = tfds.load('cityscapes', split='train', shuffle_files=True, data_dir='gtFine')
#'gdrive/MyDrive/ColabNotebooks/gtFine_trainvaltest.zip'
assert isinstance(ds, tf.data.Dataset)
print(ds)

AssertionError: ignored

In [28]:
import tensorflow_datasets as tfds
dataset = 'cityscapes'
builder = tfds.builder(dataset)

#print(tfds.download.DownloadConfig.manual_dir)
print(builder.data_path)
#builder.download_and_prepare()

/root/tensorflow_datasets/cityscapes/semantic_segmentation/1.0.0


In [15]:
!cp gdrive/MyDrive/ColabNotebooks/gtFine_trainvaltest.zip ~/tensorflow_datasets/downloads/manual/
#mv leftImg8bit_trainvaltest.zip ~/tensorflow_datasets/downloads/manual/
#/usr/local/lib/python3.10/dist-packages/tensorflow_datasets/datasets/cityscapes

/bin/bash: line 1: copy: command not found


In [13]:
!python Cityscapes.py

Traceback (most recent call last):
  File "/content/Cityscapes.py", line 6, in <module>
    from gluoncv.utils import download, makedirs
ModuleNotFoundError: No module named 'gluoncv'


In [6]:
from gluoncv.data import CitySegmentation
train_dataset = CitySegmentation(split='train')
val_dataset = CitySegmentation(split='val')
print('Training images:', len(train_dataset))
print('Validation images:', len(val_dataset))

ModuleNotFoundError: ignored

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [ ]:
ds = tfds.load('cityscapes', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

In [ ]:
#Functions to Read Image and Split Them
def _read_to_tensor(fname, output_height=256, output_width=512, normalize_data=False):
    '''Original images are 256 x 512 x 3. Left half is original image, right is semantic seg'''
    img_strings = tf.read_file(fname)
    imgs_decoded = tf.image.decode_jpeg(img_strings)
    output = tf.image.resize_images(imgs_decoded, [output_height, output_width])
    if normalize_data:
        output = (output - 128) / 128
    return output

def _get_left_img_half(inp, width=256):
    return inp[:, :width, :]

def _get_right_img_half(inp, width=256):
    return inp[:, width:, :]

In [ ]:
#Get List of Files and Apply Functions Above to Create Dataset
#img_dir = '../input/cityscapes_data/cityscapes_data/train'
img_dir = 'https://drive.google.com/file/d/1-7pEZa3O5UTDjH3L6F4ZZBjxMnRgpwEL/view?usp=sharing'
file_list = os.listdir(img_dir)
img_paths = [os.path.join(img_dir, fname) for fname in file_list]

# Start with a dataset of directory names.
output_height = 256
output_width = 256
my_data = tf.data.Dataset.from_tensor_slices(img_paths)
img_tensors = my_data.map(_read_to_tensor)
left_imgs = img_tensors.map(_get_left_img_half)
right_imgs = img_tensors.map(_get_right_img_half)

FileNotFoundError: ignored

In [ ]:
#Simple Demo That The Images Have Been Read

#left_batches = tfe.Iterator(left_imgs)  # outside of TF Eager, we would use make_one_shot_iterator
left_batches = tf.keras.preprocessing.image.Iterator(left_imgs)
#right_batches = tfe.Iterator(right_imgs)
right_batches = tf.keras.preprocessing.image.Iterator(right_imgs)
n_images_to_show = 5

for i in range(n_images_to_show):
    left_img = left_batches.next().numpy().astype(np.uint8)
    right_img = right_batches.next().numpy().astype(np.uint8)
    fig = plt.figure()
    fig.add_subplot(1,2,1)
    plt.imshow(left_img)
    fig.add_subplot(1,2,2)
    plt.imshow(right_img)
    plt.show()

NameError: ignored